**DATA CLEANING**

In [0]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,FloatType
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

import pandas as pd
import numpy as np


In [0]:
%pip install openpyxl

In [0]:
dbutils.library.restartPython()

In [0]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,FloatType
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

import pandas as pd
import numpy as np

In [0]:
##spark = SparkSession.builder.appName("DATA VALIDATION").getOrCreate()

df = pd.read_excel("/Volumes/dev/bronze/transactions/PAN Number Validation Dataset.xlsx")
print(df.head(10))
total_records = len(df)
print('Total Records= ', total_records)



In [0]:
df['Pan_Numbers'] = df['Pan_Numbers'].astype("string").str.strip().str.upper()
print(df.head(10))

In [0]:
df_m = print(df['Pan_Numbers'] == '')

In [0]:
print(df['Pan_Numbers'].isna())

In [0]:
df = df.replace({"Pan_Nubers":''},pd.NA).dropna(subset="Pan_Numbers")
#print(df[df['Pan_Numbers' == '']])
print('Total_Records ', len(df))

In [0]:

print('Total_Records =', len(df))
print('Unique Records =', df['Pan_Numbers'].nunique())

df = df.drop_duplicates(subset="Pan_Numbers", keep='first')
print('Total Records =', len(df))



**VALIDATION**

In [0]:
def adjusant_repitation(pan):
  for i in range(len(pan)-1):
    if pan[i] == pan[i+1]:
      return True
  return False

print(adjusant_repitation('LMNOP'))
print(adjusant_repitation('PQRSTT'))
print(adjusant_repitation('AABBCC'))



using list comprehension

In [0]:
def adjusant_repitation1(pan):
  return any(pan[i] == pan[i+1] for i in range(len(pan)-1))
  
print(adjusant_repitation1('LMNOP'))
print(adjusant_repitation1('PQRSTT'))
print(adjusant_repitation1('AABBCC'))

In [0]:
#check thw=e ascii val in sql
# In python 
def is_sequence(pan):
    for i in range(len(pan)-1):
        if ord(pan[i+1]) - ord(pan[i]) != 1:
            return False
    return True
print(is_sequence('LMNOP'))
print(is_sequence('PQRSTT'))
print(is_sequence('AABBCC'))

In [0]:
def is_sequence(pan):

    return all(ord(pan[i+1]) - ord(pan[i]) == 1 for i in range(len(pan)-1))

print(is_sequence('LMNOP'))
print(is_sequence('PQRSTT'))
print(is_sequence('AABBCC'))

In [0]:
def is_valid_pan(pan):
 
 if len(pan) != 10:
    return False

print(is_valid_pan('5526452344'))

In [0]:
import re

def is_valid_pan(pan):
 
 if len(pan) != 10:
    return False

 if re.match(r'^[A-Z]{5}[0-9]{4}[A-Z]$', pan):
     return True
 else:
     return False
 
df["status"] = df["Pan_Numbers"].apply((lambda x: "Valid" if is_valid_pan(x) else "Invalid"))

print(df.head(10))

print(is_valid_pan('5526452344'))

In [0]:
valid_df =(df['status'] == 'Valid').sum()
invalid_df = (df['status'] == 'Invalid').sum()
#total_records = len(df)

print(valid_df)
print(invalid_df)
misssing_cnt = total_records - (valid_df + invalid_df)
print(misssing_cnt)


In [0]:
df_summary = pd.DataFrame({"TOTAL PROCESSED RECORDS" : [total_records],
                           "TOTAL VALID RECORDS" : [valid_df],
                           "TOTAL INVALID RECORDS" : [invalid_df],
                           "TOTAL MISSING RECORDS" : [misssing_cnt]
                           })
print(df_summary.head())

In [0]:
df_summary.to_csv(
  '/Volumes/dev/bronze/transactions/pan_card_summary.csv',
  index=False
)
display(df_summary)

In [0]:
with pd.ExcelWriter('pan_card_summary.xlsx') as writer:
  df.to_excel(writer, sheet_name = "Pan valiations", index = False)
  df_summary.to_excel(writer, sheet_name = "Pan Summary", index = False)
display(df_summary)